# **📊 Project Explanation — Gamestop vs Tesla Interactive Analysis**

Compare Tesla (TSLA) and GameStop (GME) stock performance and revenue using yfinance, BeautifulSoup, and Plotly.
The notebook downloads historical stock data, scrapes company revenues from IBM Skills Network pages, cleans the data, and visualizes both price and revenue trends interactively.

# 1️⃣ Importing Libraries
We start by importing the key Python libraries used throughout the project:

In [39]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots


🔹 Purpose:

pandas → data cleaning and manipulation

yfinance → download stock market data

requests + BeautifulSoup → web scraping (extracting revenue tables)

plotly → creating interactive visualizations

These tools together create a full workflow:

Data collection → Cleaning → Visualization.

# 2️⃣ Getting Stock Data from Yahoo Finance
We define a reusable function that retrieves full historical data for any stock ticker.

In [40]:
def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    df = stock.history(period="max")
    df.reset_index(inplace=True)
    return df


Then we call it for Tesla and GameStop:

In [41]:
tesla_data = get_stock_data("TSLA")
gme_data = get_stock_data("GME")

🔹 Why?

The yfinance library gives free, reliable access to market data —
perfect for quick analysis without needing API keys.

# 3️⃣ Scraping Tesla Revenue Data
We use web scraping to get Tesla’s revenue table from an IBM project page.

In [42]:
def get_tesla_revenue():
    url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
    response = requests.get(url, timeout=30)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find_all("table")[1]
    rows = table.find("tbody").find_all("tr")

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            data.append((date, revenue))

    df = pd.DataFrame(data, columns=["Date", "Revenue"])
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    return df.dropna()


# 4️⃣ Scraping GameStop Revenue Data

Same approach, but the table structure is slightly different.

In [43]:
def get_gme_revenue():
    url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
    response = requests.get(url, timeout=30)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    tbodies = soup.find_all("tbody")
    if len(tbodies) < 2:
        raise ValueError("Could not find GameStop revenue table.")

    tbody = tbodies[1]
    rows = tbody.find_all("tr")

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace("$", "").replace(",", "")
            data.append((date, revenue))

    df = pd.DataFrame(data, columns=["Date", "Revenue"])
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    return df.dropna()


# 5️⃣ Data Cleaning

Remove dollar signs $ and commas ,

Convert revenue values to numeric

Convert date strings to proper datetime format

Drop invalid or empty rows

✅ This ensures all columns are consistent and can be plotted without errors.

# 6️⃣ Visualizing with Plotly

We create a single, reusable function to plot both stock price and revenue together.

In [44]:
def make_graph(stock_data, revenue_data, stock_name):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Share Price","Revenue"),
                        vertical_spacing=0.3)
    fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Close"], name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=revenue_data["Date"], y=revenue_data["Revenue"], name="Revenue"), row=2, col=1)
    fig.update_layout(height=900, title=stock_name, showlegend=False)
    fig.show()


In [45]:
tesla_data = get_stock_data("TSLA")
tesla_revenue = get_tesla_revenue()
make_graph(tesla_data, tesla_revenue, "Tesla")

In [46]:
gme = get_stock_data("GME")
gme_rev = get_gme_revenue()
make_graph(gme, gme_rev, "GameStop")